In [ ]:
import pandas as pd

## 데이터셋 불러오기

In [ ]:
train = pd.read_csv('./data/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./data/test.csv', encoding = 'utf-8-sig')

In [ ]:
print(len(train))
print(len(test))

In [ ]:
train.head(20)

In [ ]:
train['input'][0]

In [ ]:
train['output'][0]

## 형태소 기반 단어 사전 생성

In [ ]:
from kiwipiepy import Kiwi
from collections import defaultdict

kiwi = Kiwi()
obfuscation_vocab = defaultdict(set)

for idx, row in train.iterrows():
    original_text = row['output']
    obfuscated_text = row['input']
    obfuscated_chars = list(obfuscated_text)

    tokens = kiwi.tokenize(original_text)

    for token in tokens:
        if token.len != 1:
            obfuscated_noun = ''.join(obfuscated_chars[token.start:token.start+token.len])
            obfuscation_vocab[token.form].add(obfuscated_noun)

In [ ]:
print("명사와 대응되는 난독화된 단어 사전:")
for noun, obfuscated_list in obfuscation_vocab.items():
    print(f"{noun}: {obfuscated_list}")

In [ ]:
test = pd.read_csv("./data/test.csv", encoding='utf-8-sig')

In [ ]:
test.head()

In [ ]:
for _, row in test.iterrows():
    for noun, obfuscated_list in obfuscation_vocab.items():
        for obfuscated_word in obfuscated_list:
            if obfuscated_word in row['input']:
                print("변경 전 : ", row['input'])
                print(f"변경된 단어 : {obfuscated_word} -> {noun}")
                row['input'] = row['input'].replace(obfuscated_word, noun)
    print("변경 후 : ", row['input'])

## 전처리된 데이터 저장

In [ ]:
test.to_csv("./data/test_preprocessed.csv", index=False, encoding='utf-8-sig')